In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
# Create some data using linear regression formula of y = mx + c
weight = 0.7
bias = 0.3

# Create range values
start = 0
end = 1
step = 0.02

# Create X and Y features
X = torch.arange(start,end,step).unsqueeze(dim=1)
Y = weight * X + bias

# Split data
train_split = int(0.8 * len(X))
X_train, Y_train = X[:train_split], Y[:train_split]
X_test, Y_test, = X[train_split:], Y[train_split:]

# Plot the data
def plot_predictions(train_data=X_train, train_labels=Y_train, test_data=X_test, test_labels=Y_test, predictions=None):
  """
  Plots training data, test data, and compares predictions.
  """

  plt.figure(figsize=(10,7))
  # Plot training data in blue
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training Data")

  # Plot test data in green
  plt.scatter(test_data, test_labels, c="g", s=4, label="Testing Data")
  if predictions is not None:
    plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

  # Show the legend
  plt.legend(prop={"size": 14})

In [2]:
# Building a PyTorch Linear Model
class LinearRegressionModelV2(nn.Module):
  def __init__(self):
    super().__init__()

    self.linear_layer = nn.Linear(in_features=1, out_features=1)

  def forward(self,x: torch.Tensor) -> torch.Tensor:
    return self.linear_layer(x)

torch.manual_seed(42)
model_1 = LinearRegressionModelV2()

In [3]:
# Set the model to use a target device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_1.to(device)
next(model_1.parameters()).device

device(type='cuda', index=0)

In [4]:
# Training Code
# 1. Setup Loss Function and Optimizer

loss_fn = nn.L1Loss() # Same as MAE
optimizer = torch.optim.SGD(params=model_1.parameters(),lr=0.01)

In [7]:
torch.manual_seed(42)

epochs = 200
# Put data on the target device
X_train = X_train.to(device)
Y_train = Y_train.to(device)
X_test = X_test.to(device)
Y_test = Y_test.to(device)
for epoch in range(epochs):
  model_1.train()

  # 1. Forward Pass
  Y_pred = model_1(X_train)

  # 2. Calculate the loss
  loss = loss_fn(Y_pred, Y_train)

  # 3. Optimizer zero gradient
  optimizer.zero_grad()

  # 4. Perform backpropagation
  loss.backward()

  # 5. Optimizer step
  optimizer.step()

  # Testing
  model_1.eval()
  with torch.inference_mode():
    test_preds = model_1(X_test)
    test_loss = loss_fn(test_preds, Y_test)

    # Print out what's happening

    if epoch % 50 == 0:
      print(f"Epoch: {epoch} | Loss: {loss} | Test Loss: {test_loss}")



Epoch: 0 | Loss: 0.0012645035749301314 | Test Loss: 0.013801801018416882
Epoch: 50 | Loss: 0.0012645035749301314 | Test Loss: 0.013801801018416882
Epoch: 100 | Loss: 0.0012645035749301314 | Test Loss: 0.013801801018416882
Epoch: 150 | Loss: 0.0012645035749301314 | Test Loss: 0.013801801018416882


In [ ]:
with torch.inference_mode():
  preds=model_1(X_test).cpu()

plot_predictions(predictions=preds)

In [17]:
# Save model
from os import mkdir
import os
from pathlib import Path

model_folder = Path("models")
model_folder.mkdir(parents=True, exist_ok=True)


model_location = f"{model_folder}{os.sep}pytorch_workflow_model_1_linear.pth"
torch.save(f=model_location, obj=model_1.state_dict())

In [18]:
# Load model
loaded_model_1 = LinearRegressionModelV2()
loaded_model_1.load_state_dict(torch.load(model_location))

loaded_model_1.to(device)

LinearRegressionModelV2(
  (linear_layer): Linear(in_features=1, out_features=1, bias=True)
)

In [19]:
next(loaded_model_1.parameters())

Parameter containing:
tensor([[0.6968]], device='cuda:0', requires_grad=True)

In [33]:
# Evaluate the loaded model.
loaded_model_1.eval()
with torch.inference_mode():
  loaded_model_1_preds = loaded_model_1(X_test)
predictions = predictions.to(device)

predictions == loaded_model_1_preds


tensor([[True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True]], device='cuda:0')